In [1]:
# -*- coding: utf-8 -*-
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, Flatten
from keras.models import Model, load_model
from keras.callbacks import TensorBoard
from keras.losses import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
import pickle
import datetime
import numpy as np
import pandas as pd
import seaborn as sns


Using TensorFlow backend.


## Initialize Variables

In [2]:
numericColumns = ['MIN_PS', 'MAX_PS', 'AVG_PS', 'STD_DEV_PS', 'FLOW_DURATION', 'MIN_PIAT', 'MAX_PIAT', 'AVG_PIAT', 'STD_DEV_PIAT', 'PKT_TOTAL_COUNT', 'OCTET_TOTAL_COUNT'
             , 'F_MIN_PS', 'F_MAX_PS', 'F_AVG_PS', 'F_STD_DEV_PS', 'F_FLOW_DURATION', 'F_MIN_PIAT', 'F_MAX_PIAT', 'F_AVG_PIAT', 'F_STD_DEV_PIAT', 'F_PKT_TOTAL_COUNT', 'F_OCTET_TOTAL_COUNT'
             , 'B_MIN_PS', 'B_MAX_PS', 'B_AVG_PS', 'B_STD_DEV_PS', 'B_FLOW_DURATION', 'B_MIN_PIAT', 'B_MAX_PIAT', 'B_AVG_PIAT', 'B_STD_DEV_PIAT', 'B_PKT_TOTAL_COUNT', 'B_OCTET_TOTAL_COUNT']


In [3]:
pd.set_option("display.max_columns", 101)
pickle_path = 'data/dns-flow.pkl'
numericColumns = ['MIN_PS', 'MAX_PS', 'AVG_PS', 'STD_DEV_PS', 'FLOW_DURATION', 'MIN_PIAT', 'MAX_PIAT', 'AVG_PIAT', 'STD_DEV_PIAT', 'PKT_TOTAL_COUNT', 'OCTET_TOTAL_COUNT'
             , 'F_MIN_PS', 'F_MAX_PS', 'F_AVG_PS', 'F_STD_DEV_PS', 'F_FLOW_DURATION', 'F_MIN_PIAT', 'F_MAX_PIAT', 'F_AVG_PIAT', 'F_STD_DEV_PIAT', 'F_PKT_TOTAL_COUNT', 'F_OCTET_TOTAL_COUNT'
             , 'B_MIN_PS', 'B_MAX_PS', 'B_AVG_PS', 'B_STD_DEV_PS', 'B_FLOW_DURATION', 'B_MIN_PIAT', 'B_MAX_PIAT', 'B_AVG_PIAT', 'B_STD_DEV_PIAT', 'B_PKT_TOTAL_COUNT', 'B_OCTET_TOTAL_COUNT']



## load model and min-max scaler

load exists autoencoder model and min-max scaler

In [22]:
minMaxScaler = pickle.load(open('../analyze/model/{}'.format('minMaxScalerForDnsFlow-14.02.19_15:10'), 'rb'))
#xScaled = minMaxScaler.fit_transform(datas[numericColumns])
#x = pd.DataFrame(xScaled)
trainedAutoencoder = load_model('../analyze/model/{}'.format('autoencoderForDnsFlow-14.02.19_15:10'))
#xPred = trainedAutoencoder.predict(x)
#mse_score = mse(xScaled, xPred)

## Data Pre-processing

In [23]:
malware_datas = pd.read_pickle('../data/dns-malware-flow.pkl')
normal_datas = pd.read_pickle('../data/dns-flow.pkl')

In [24]:
datas = pd.concat([malware_datas, normal_datas])

In [25]:
datas.info()
# normal : 100,000
# malware : 2,000
# Q1. Do we need to equalize the number of flow at the same count?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107529 entries, 0 to 105034
Data columns (total 39 columns):
AVG_PIAT               107529 non-null float64
AVG_PS                 107529 non-null float64
B_AVG_PIAT             107529 non-null float64
B_AVG_PS               107529 non-null float64
B_FLOW_DURATION        107529 non-null float64
B_MAX_PIAT             107529 non-null float64
B_MAX_PS               107529 non-null float64
B_MIN_PIAT             107529 non-null float64
B_MIN_PS               107529 non-null float64
B_OCTET_TOTAL_COUNT    107529 non-null float64
B_PKT_TOTAL_COUNT      107529 non-null float64
B_STD_DEV_PIAT         107529 non-null float64
B_STD_DEV_PS           107529 non-null float64
DST_IP                 107529 non-null object
DST_PORT               107529 non-null object
FLOW_DURATION          107529 non-null float64
F_AVG_PIAT             107529 non-null float64
F_AVG_PS               107529 non-null float64
F_FLOW_DURATION        107529 non-null float6

In [27]:
min_max_scaled_datas = minMaxScaler.fit_transform(datas[numericColumns])

In [29]:
original_datas = datas.copy()

In [30]:
datas[numericColumns] = min_max_scaled_datas

In [31]:
original_datas.describe()

,AVG_PIAT,AVG_PS,B_AVG_PIAT,B_AVG_PS,B_FLOW_DURATION,B_MAX_PIAT,B_MAX_PS,B_MIN_PIAT,B_MIN_PS,B_OCTET_TOTAL_COUNT,B_PKT_TOTAL_COUNT,B_STD_DEV_PIAT,B_STD_DEV_PS,FLOW_DURATION,F_AVG_PIAT,F_AVG_PS,F_FLOW_DURATION,F_MAX_PIAT,F_MAX_PS,F_MIN_PIAT,F_MIN_PS,F_OCTET_TOTAL_COUNT,F_PKT_TOTAL_COUNT,F_STD_DEV_PIAT,F_STD_DEV_PS,MAX_PIAT,MAX_PS,MIN_PIAT,MIN_PS,OCTET_TOTAL_COUNT,PKT_TOTAL_COUNT,STD_DEV_PIAT,STD_DEV_PS
count,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000
mean,160.116461,131.987474,446.510519,197.823606,714.048478,533.131492,219.179710,365.852012,177.917818,312.218694,1.492779,75.295005,18.607357,714.791187,437.510793,67.603648,714.720874,533.665990,69.350947,353.280041,66.044007,103.873262,1.536609,80.587886,1.489972,533.671470,219.241832,0.046771,66.044007,416.091957,3.029387,230.487652,69.450468
std,302.889127,48.791327,850.890383,98.116052,1522.391089,998.043255,118.095002,788.300000,95.309443,300.509971,0.958666,253.323541,38.134197,1522.848572,840.619396,8.111017,1522.849878,998.319678,9.708315,779.468279,8.270017,72.159322,1.055003,263.363261,3.353916,998.259518,117.997728,0.102713,8.270017,362.781722,1.992563,430.417921,51.589428
min,0.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.000000,0.000000,0.000000,34.000000,0.000000,34.000000,34.000000,1.000000,0.000000,0.000000,0.000000,34.000000,0.000000,34.000000,52.000000,1.000000,0.000000,0.000000
25%,0.022127,97.000000,0.000000,126.750000,0.000000,0.000000,129.000000,0.000000,109.000000,130.000000,1.000000,0.000000,0.000000,0.022145,0.000000,62.000000,0.000000,0.000000,63.000000,0.000000,61.000000,64.000000,1.000000,0.000000,0.000000,0.022139,129.000000,0.016402,61.000000,200.000000,2.000000,0.000000,30.000000
50%,0.038522,119.500000,0.000000,171.000000,0.000000,0.000000,193.000000,0.000000,148.000000,221.000000,1.000000,0.000000,0.000000,0.038568,0.000000,67.000000,0.000000,0.000000,68.000000,0.000000,65.000000,71.000000,1.000000,0.000000,0.000000,0.038541,193.000000,0.024851,65.000000,292.000000,2.000000,0.000000,58.156876
75%,172.062392,155.250000,494.433291,246.000000,581.833126,559.319280,279.000000,173.410076,228.000000,375.000000,2.000000,0.000000,14.500000,586.690593,469.455374,71.500000,586.656635,563.019802,73.000000,113.596478,70.000000,129.000000,2.000000,0.000000,1.479020,562.977237,279.000000,0.036830,70.000000,483.000000,4.000000,253.220824,98.500000
max,1790.303694,375.000000,3599.901488,687.000000,13748.010257,5499.780357,725.000000,3599.901488,687.000000,4415.000000,16.000000,1780.190331,296.040854,13748.029622,3599.575149,176.000000,13748.015712,3600.057741,176.000000,3599.575149,176.000000,1238.000000,18.000000,1798.965481,51.000000,3599.977986,725.000000,4.861821,176.000000,5398.000000,34.000000,1790.291211,312.000000


In [32]:
datas.describe()

,AVG_PIAT,AVG_PS,B_AVG_PIAT,B_AVG_PS,B_FLOW_DURATION,B_MAX_PIAT,B_MAX_PS,B_MIN_PIAT,B_MIN_PS,B_OCTET_TOTAL_COUNT,B_PKT_TOTAL_COUNT,B_STD_DEV_PIAT,B_STD_DEV_PS,FLOW_DURATION,F_AVG_PIAT,F_AVG_PS,F_FLOW_DURATION,F_MAX_PIAT,F_MAX_PS,F_MIN_PIAT,F_MIN_PS,F_OCTET_TOTAL_COUNT,F_PKT_TOTAL_COUNT,F_STD_DEV_PIAT,F_STD_DEV_PS,MAX_PIAT,MAX_PS,MIN_PIAT,MIN_PS,OCTET_TOTAL_COUNT,PKT_TOTAL_COUNT,STD_DEV_PIAT,STD_DEV_PS
count,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000,107529.000000
mean,0.089435,0.287353,0.124034,0.287953,0.051938,0.096937,0.302317,0.101628,0.258978,0.070718,0.093299,0.042296,0.062854,0.051992,0.121545,0.236645,0.051987,0.148238,0.248950,0.098145,0.225662,0.058034,0.031565,0.044797,0.029215,0.148243,0.268078,0.009620,0.225662,0.068105,0.061497,0.128743,0.222598
std,0.169183,0.143083,0.236365,0.142818,0.110735,0.181470,0.162890,0.218978,0.138733,0.068066,0.059917,0.142301,0.128814,0.110768,0.233533,0.057120,0.110769,0.277307,0.068368,0.216545,0.058240,0.059933,0.062059,0.146397,0.065763,0.277296,0.170764,0.021126,0.058240,0.067860,0.060381,0.240418,0.165351
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000012,0.184751,0.000000,0.184498,0.000000,0.000000,0.177931,0.000000,0.158661,0.029445,0.062500,0.000000,0.000000,0.000002,0.000000,0.197183,0.000000,0.000000,0.204225,0.000000,0.190141,0.024917,0.000000,0.000000,0.000000,0.000006,0.137482,0.003374,0.190141,0.027684,0.030303,0.000000,0.096154
50%,0.000022,0.250733,0.000000,0.248908,0.000000,0.000000,0.266207,0.000000,0.215429,0.050057,0.062500,0.000000,0.000000,0.000003,0.000000,0.232394,0.000000,0.000000,0.239437,0.000000,0.218310,0.030731,0.000000,0.000000,0.000000,0.000011,0.230101,0.005111,0.218310,0.044893,0.030303,0.000000,0.186400
75%,0.096108,0.355572,0.137346,0.358079,0.042321,0.101698,0.384828,0.048171,0.331878,0.084938,0.125000,0.000000,0.048980,0.042675,0.130420,0.264085,0.042672,0.156392,0.274648,0.031558,0.253521,0.078904,0.058824,0.000000,0.029000,0.156384,0.354559,0.007575,0.253521,0.080621,0.090909,0.141441,0.315705
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Analyze

by using predict_x and original x, let's figure out which feature has more important weight in auto encoder

In [8]:
mse_score = x.iloc[:,-1:]

In [11]:
x.describe()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
count,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000
mean,0.698703,0.719102,0.756883,0.628878,0.025303,0.010845,0.029447,0.029081,0.029159,0.163077,0.191876,0.698703,0.707559,0.703131,0.012554,0.025301,0.028189,0.029444,0.028808,0.001504,0.023924,0.088025,0.843291,0.846571,0.844923,0.009023,0.025300,0.028189,0.029444,0.028808,0.001504,0.267442,0.259083
std,0.114357,0.076420,0.097626,0.059559,0.114864,0.020514,0.132796,0.131359,0.131636,0.079034,0.074257,0.114357,0.113830,0.111987,0.061880,0.114864,0.129500,0.132797,0.130517,0.030789,0.093976,0.087155,0.047801,0.046757,0.046470,0.048190,0.114864,0.129500,0.132797,0.130517,0.030789,0.069354,0.068058
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.652174,0.659341,0.694118,0.580952,0.000002,0.009842,0.000003,0.000008,0.000000,0.142857,0.166461,0.652174,0.652174,0.652174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061224,0.814371,0.814371,0.814371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.233276
50%,0.695652,0.703297,0.752941,0.600000,0.000002,0.010050,0.000003,0.000009,0.000000,0.142857,0.173859,0.695652,0.695652,0.695652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065306,0.832335,0.838323,0.832335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.240137
75%,0.782609,0.791209,0.835294,0.695238,0.000002,0.010280,0.000003,0.000009,0.000000,0.142857,0.181258,0.782609,0.782609,0.782609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.073469,0.880240,0.886228,0.885479,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.253859
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
%config InlineBackend.figure_format = 'png' 